# **0 写在开始之前**

> - ***课题***: Housing Price Analysis —— 杨浦区二手房房价分析
> - ***作者***: 杜圣辉 2017111136
> - ***班级***: 17级信息管理于信息系统1班
> - ***课程***: 数据科学导论
> - ***日期***: 2020年4月-6月

## **0.1 注释与引用**
- **注释**：正如报告中所说的一样,我会在脚本中把一切有必要的注释都写上来便于助教学长或韩老师阅读;
- **引用**：我用的编辑器是 [jupyter lab](https://jupyterlab.readthedocs.io/en/stable/)，在一些文件引用的格式上可能和其他编辑器有出入，请根据您的需要更改地址； 

## **0.2 关于爬虫工具**
- 这里我用了selenium工具包:
    - 它在爬取过程中会对一个浏览器对象进行自动化操作能够高度模仿我们日常的浏览行为(必要时候也需要更改请求头和IP,但本次不会涉及);
    - 其操作和web开发时候很类似,通过查询元素的标签类型(a,p,div等),id,class,xpath等来完成对网页的操作,同时可以通过对文本框的 clear(), send_keys(), click()等方法来完成一些基本的表单填写和交互;
    - 关于详细的上手,可以参考这篇文章:
    - 但是在一切开始之前,若要调试代码,可以安装您谷歌浏览器对应的chromedirver,并将应用程序的地址将我的覆盖:
        - [ChromeDriver与Chrome版本对应参照表及ChromeDriver下载链接](https://blog.csdn.net/BinGISer/article/details/88559532);

## **0.3 关于存储**
- **CSV FILE**：
    - 在数据存储方面，为了方便展示，我会暂且保存为csv文件；
- **MySQL**：
    - 有关于数据库的存储操作，我会在代码块中注释掉；
    - 我会确保所有的对标操作都是<u>可以执行的脚本代码</u>；
    - 这里会涉及到另外两个脚本文件，三者会互相协同完成对表操作，分别为：
        - [mydb.py](mydb.py): 我尝试通过编写数据库查询类，来完成基础的对表操作；
        - [sql.py](sql.py): 存放mydb.py中所需的sql脚本；

## **0.4 关于结构**
- 本脚本会集中于数据爬取、存储、*初步预处理*
- 进一步的数据预处理可以移步[Attemptation.ipynb]()和[Further_Analysis.ipynb]()部分：
    - **Attemptation**: 包含我根据已有字段进行初步的数据分析；
    - **Further Analysis**：已有字段中尽管有房屋周边信息（学校、基础设施、交通等），但是不够完整，于是我寻求百度地图（API无法精确，我使用的是坐标拾取系统）完成进一步的数据处理，并将舒适步行距离内的各种设施进行统计，完成进一步的特征构建，并带入回归模型中进行计算；

# **1 初始化**

首先需要引入相关的工具包、定义爬取数据的边界

In [1]:
import selenium
from selenium import webdriver
import pandas as pd
import time, random
import numpy as np
import warnings
warnings.filterwarnings('ignore')

options=webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors') # 类似于warnings中的filterwarnings('ignore')

"""注意：这里需要把chromedriver的路径修改为您电脑里安装的chromedriver的路径"""
chromedriver = 'C:/Users/Dushenghui/PycharmProjects/dailypy/venv/chromedriver.exe'
# phantomjs = 'C:/Users/Dushenghui/PycharmProjects/dailypy/venv/phantomjs-2.1.1-windows/bin/phantomjs' # 当然这里也可以用PhantomJS

base_url = "https://shanghai.anjuke.com/sale/" # 基础url
antispam = "https://nanjing.anjuke.com/antispam-block/?from=antispam" # 反爬虫链接，匹配，如果进入这个网站就退出爬虫

# 如果需要更多的数据，可以去尝试把外部大循环的列表扩充为这里的字典
# dict_district = ['pudong', 'minhang', 'baoshan', 'xuhui', 'songjiang', 'jiading',\
#                  'jingan', 'putuo', 'yangpu', 'hongkou', 'changning', 'huangpu', \
#                  'qingpu', 'fengxian', 'chongming', 'shanghaizhoubian']

# 为了方便起见，同时为了能够更加细化模型，这里选择爬取杨浦区的数据
dict_district1 = ['yangpu']

# 初始化driver对象
driver = webdriver.Chrome(executable_path=chromedriver, chrome_options=options)
# driver = webdriver.PhantomJS(executable_path=phantomjs)  # Phantom的初始化方法亦然

# 函数工具1：确保我得到的结果是对应XPATH中的文本或空值
#   - 方便后续数据处理（正则匹配 or 填充缺失值）
#   - 这是由于网页本身的元素导致的，有些信息可能不会存在，按照XPATH或者CLASS寻找都会出现这样的问题
def get_my_page_elem_by_xpath(browser, elem_xpath):
    """
        get_my_page_elem_by_xpath： 根据输入的elem_xpath，再browser中返回对应路径中存储的文本
            - browser：selenium.webdriver()
            - elem_xpath: string
    """
    try:
        # find element, if not found in the page, return null instead
        return browser.find_element_by_xpath(elem_xpath).text
    except:
        try:
            return browser.find_element_by_xpath(elem_xpath)
        except:
            return None

# **2 爬取**

这里通过三个循环完成爬取:
- 循环1: 对区循环(目前只有杨浦,但如果要扩大数据集范围可以额外加上别的区);
- 循环2: 对页循环(完成对一个区下所有页的遍历,值得注意的是这里安居客所展示的最大页数为50页,同豆瓣类似);
- 循环3: 对房循环(这里的房指的是一页下的房源)

另外,数据存储会在爬完之后进行操作

In [ ]:
lst = []

"""【外部循环】：实现对不同_区_的遍历"""
for item_district in dict_district1:
    # todo：得到当前区爬虫的url，加上格式化输入的空间，实现对每一页的遍历爬取
    #     - current_district: 形如 https://shanghai.anjuke.com/sale/pudong/p{0}
    current_district = base_url + str(item_district) + '/p{0}'
    
    # 设置最开始的页数
    #     - 这个循环只有当前区爬取完毕后才会停止
    current_page_number = 1
    
    """【内部循环】：实现对同一区内不同_页_的遍历"""
    while True:
        # current_page: 形如 https://shanghai.anjuke.com/sale/pudong/p1
        current_page = current_district.format(current_page_number)
        driver.get(current_page)
        print(driver.current_url)
        # todo：检验，如果当前的url同base相同的情况：
        #     - 也就是说，超出了本区最大页数，安居客会自动返回到base；
        #     - 表明本区爬取完毕，进入下一个区； 
        if driver.current_url == base_url:
            break

        if 'antispam' in (driver.current_url).lower() or 'ant-ispam' in (driver.current_url).lower():
            break

        # todo：此时仍然在本区中，可以继续爬取数据
        else:
            # todo: 得到当前页所有房屋信息的div，作为获得每一套房屋连接的基础
            house_elems = driver.find_elements_by_class_name('houseListTitle')
            
            for item_house in house_elems:
                
                if driver.current_url == antispam:
                    break
                
                item_house_url = item_house.get_attribute('href') # 得到每个url
                js_open_item_house = 'window.open("{0}");'.format(str(item_house_url)) # 讲网页放入JavaScript指令当中，便于之后打开操作
                driver.execute_script(js_open_item_house) # 打开新的网页

                ori_page = driver.window_handles[0] # 变量存放原本的第一页
                new_page = driver.window_handles[-1] # 变量存放新打开的一页
                
                # todo: 将selenium对象切换到新的页面中
                driver.switch_to_window(new_page) # selenium 的特性，虽然打开了一个新的tab，但是并不会立刻将状态默认转移到那个tab中去

                print(driver.current_url)
                
                # 房屋标题
                title = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[2]/h3')
                # 房屋面积
                house_total_price = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[1]/div[1]/span[1]/em')
                # 房屋信息
                info_detail = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[3]/div/div[1]/ul') 
                # 小区总面积
                comm_area = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[6]/div[2]/div[1]/p[2]')
                # 小区总户数
                comm_households = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[6]/div[2]/div[2]/p[2]/text()')
                # 小区容积率
                comm_plot_ratio = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[6]/div[2]/div[3]/p[2]')
                # 小区停车位数
                comm_parking_num = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[6]/div[2]/div[4]/p[2]/text()')
                # 小区绿化率
                comm_green_ratio = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[6]/div[2]/div[5]/p[2]')
                # 小区物业费
                comm_property_fee = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[6]/div[2]/div[6]/p[2]')
                # 专家解读优点
                adv = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[7]/ul/li/div[1]/dl[1]')
                # 专家解读不足
                disadv = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[7]/ul/li/div[1]/dl[2]')
                # 核心卖点
                core_point = get_my_page_elem_by_xpath(driver, '//*[@id="content"]/div[3]/div[1]/div[3]/div/div[2]/div[1]/div/span')
                

                lst.append([title, house_total_price, info_detail, comm_area, comm_households, comm_plot_ratio, comm_parking_num, \
                            comm_green_ratio, comm_property_fee, item_district, adv, disadv, core_point, driver.current_url])
                
                print(item_district, ' p', current_page_number, ': ', lst[-1])
                # todo: 令selenium对象：
                #     - 关闭打开的房屋页面，并
                #     - 切回到初始页面中；
                driver.close()
                driver.switch_to_window(ori_page)
                time.sleep(random.random()*3)
                
                # break
                # if len(lst)==15:
                    # break

            current_page_number += 1
            # break
        # break

# **3 数据清洗和预处理**

In [ ]:
# 这里进行分块，是为了防止数据保存出错，而导致需要重新爬取

house = pd.DataFrame(
    lst, 
    columns = ['title', 'house_price', 'house_info', 'comm_area', 'comm_households', 'comm_plot_ratio', 'comm_parking_num', \
              'comm_green_ratio', 'comm_property_fee', 'item_district', 'adv', 'disadv', 'core_point', 'url']
)

# house.to_csv('anjuke_house.csv', index=False)

In [1]:
from mydb import mydb # 我自己编写的数据库操作包，核心是基于pymysql，根据需求进行方法添加即可
import selenium
from selenium import webdriver
import pandas as pd
import time, random
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 工具2：展示df的基本属性
def show_df(df):
    return pd.DataFrame({
                'col' : df.columns.tolist(),
                'dtype' : [df[item].dtype for item in df.columns.tolist()],
                'uniqueVals' : [df[item].unique().shape[0] for item in df.columns.tolist()],
                'missingVals' : [df[item].isnull().sum() for item in df.columns.tolist()]
            }).sort_values(by='missingVals', ascending=False)

house=pd.read_csv('input/anjuke_house.csv')

In [2]:
print(house.shape)
house.head(2)

(3084, 14)


,title,house_price,house_info,comm_area,comm_households,comm_plot_ratio,comm_parking_num,comm_green_ratio,comm_property_fee,item_district,adv,disadv,core_point,url
0,业主是我老客户，跟我定好一手房，急需首付，急售，随时看房,350.0,所属小区：\n翔顺公寓\n房屋户型：\n2室 2厅 1卫\n房屋单价：\n48611 元/m...,暂无,NaN,1.8,NaN,25%,0.60元/m²,yangpu,特色\n【小区户型】小区主要是以多层为主近83栋，一房朝南，50-60平。两房以双南厅朝北为...,不足\n距离地铁稍微远一点 车位有点紧张 绿化稍微少点,房源标签\n售房详情\n业主是置换的，诚意出售。看房提前约，**无要求。希望能尽量快**。\...,https://shanghai.anjuke.com/prop/view/A5071614...
1,杨浦区低于市场的双南两房！热门小区！抢手户型中间楼层,285.0,所属小区：\n三门路510弄小区\n房屋户型：\n2室 1厅 1卫\n房屋单价：\n4260...,58000m²,NaN,1.86,NaN,25%,0.84元/m²,yangpu,特色\n【小区户型】小区栋数较多，车位较少。小区户型大多都在40-70平方左右。分别是南北户...,不足\n小区房龄较老，小区车位较少。,该小区位于三门路上，属于老公房小区！\n两房的主力户型为54㎡~66㎡，得房率高！\n该房户...,https://shanghai.anjuke.com/prop/view/A5084124...


In [3]:
# 查找函数，这里可以确保会返回一个确定的值而不会报错，接下来的查找步骤都会基于这个函数
def get_re_elem(pattern, string):
    import re
    try:
        return re.findall(pattern, string, re.S)[0]
    except:
        return np.NaN

house['comm_name'] = house['house_info'].apply(lambda x: get_re_elem(r'所属小区：\n(.*?)\n', x))

house = house[house['comm_name'].notnull()]

In [4]:
house = house[house['house_info'].notnull()] # 在进一步分析中，很多的特征基于info，因此将这里为空的那些去除

# 同时，这里需要对数据进行去重处理，这是因为有些同一房源会找多个中介、用统一的话术进行发布，也可能是同一个中介屡次发布一样的房源导致的
# 这时候只需要用title和info字段共同去重就可以了：
house = house.drop_duplicates(subset=['title','house_info']).reset_index(drop=True)
print(house.shape)
show_df(house)

(2141, 15)


,col,dtype,uniqueVals,missingVals
4,comm_households,float64,1,2141
6,comm_parking_num,float64,1,2141
10,adv,object,1247,175
11,disadv,object,1185,175
3,comm_area,object,167,11
5,comm_plot_ratio,object,73,11
7,comm_green_ratio,object,35,11
8,comm_property_fee,object,104,11
0,title,object,2055,4
12,core_point,object,1877,3


观测数目从3084被删减了2141

接下来进行数据存储

In [5]:
house_table = house[['title', 'house_price', 'house_info', 'adv', 'disadv', \
                     'core_point', 'comm_name', 'url']].reset_index()
print(house_table.shape)
house_table.to_csv('input/house_table.csv', index = False)
house_table.head(2)



(2141, 9)


,index,title,house_price,house_info,adv,disadv,core_point,comm_name,url
0,0,业主是我老客户，跟我定好一手房，急需首付，急售，随时看房,350.0,所属小区：\n翔顺公寓\n房屋户型：\n2室 2厅 1卫\n房屋单价：\n48611 元/m...,特色\n【小区户型】小区主要是以多层为主近83栋，一房朝南，50-60平。两房以双南厅朝北为...,不足\n距离地铁稍微远一点 车位有点紧张 绿化稍微少点,房源标签\n售房详情\n业主是置换的，诚意出售。看房提前约，**无要求。希望能尽量快**。\...,翔顺公寓,https://shanghai.anjuke.com/prop/view/A5071614...
1,1,杨浦区低于市场的双南两房！热门小区！抢手户型中间楼层,285.0,所属小区：\n三门路510弄小区\n房屋户型：\n2室 1厅 1卫\n房屋单价：\n4260...,特色\n【小区户型】小区栋数较多，车位较少。小区户型大多都在40-70平方左右。分别是南北户...,不足\n小区房龄较老，小区车位较少。,该小区位于三门路上，属于老公房小区！\n两房的主力户型为54㎡~66㎡，得房率高！\n该房户...,三门路510弄小区,https://shanghai.anjuke.com/prop/view/A5084124...


In [6]:
comm_table = house[['comm_name', 'comm_area', 'comm_households', 'comm_plot_ratio', 'comm_parking_num', 'comm_green_ratio', 'comm_property_fee']].drop_duplicates(subset=['comm_name']).reset_index(drop=True)

print(comm_table.shape)
comm_table.to_csv('input/comm_table.csv', index = False)
comm_table.head()

(563, 7)


,comm_name,comm_area,comm_households,comm_plot_ratio,comm_parking_num,comm_green_ratio,comm_property_fee
0,翔顺公寓,暂无,NaN,1.8,NaN,25%,0.60元/m²
1,三门路510弄小区,58000m²,NaN,1.86,NaN,25%,0.84元/m²
2,腾越路465弄小区,暂无,NaN,暂无,NaN,暂无,0.60元/m²
3,国和二村,38400m²,NaN,2,NaN,40%,0.55元/m²
4,开鲁五村,36000m²,NaN,1.5,NaN,25%,0.55元/m²


In [7]:
from mydb import *

host = "localhost"
user = "root"
password = "123"
database = "py_database_anjuke"

db = mydb(host, user, password, database)
db.anjuke_data_initialize() # 用来初始化数据库的表单的列

Clear Table house Success
Clear Table community Success
Initialize Success


In [10]:
db.insert(table_name_indb='COMMUNITY', pd_table=comm_table)

Finish Initialize COMMUNITY


In [11]:
db.insert(table_name_indb='HOUSE', pd_table=house_table)

Finish Initialize HOUSE
